### Cleaning/Processing Steps
The data will be demeaned similar to Zali et al., using a more conservative downsample rate of 20 Hz, given volcanic tremor is typically between 1-9 Hz.

*Read in mseed files <br>
*Check all are the same length <br>
*Demean and detrend <br>
*Anti-alias filter and downsample data to 8 Hz from 100 Hz

In [1]:
import warnings

# Suppress the specific warning from obspy
warnings.filterwarnings(
    "ignore",
    message="The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.*",
    category=UserWarning,
    module="obspy.io.mseed.core"
)


In [ ]:
import os
from obspy import read
import glob
import numpy as np

# Define the folder paths
input_folder = os.getcwd() + '/data/raw'
processed_folder = os.getcwd() + '/data/processed'
os.makedirs(processed_folder, exist_ok=True)  # Create output directory if it doesn't exist

# Target sampling rate after downsampling
target_sampling_rate = 8  # Hz

length = np.array([])

# Process each mseed file in the folder
for file_path in glob.glob(f"{input_folder}/*.mseed"):
    
    # Read the file
    st = read(file_path)
    
    # Demean, detrend, and downsample each trace
    for tr in st:
        tr.detrend("demean")   # Remove mean
        tr.detrend("linear")   # Remove linear trend
        tr.filter("lowpass", freq=target_sampling_rate / 2)  # Anti-aliasing filter
        tr.resample(target_sampling_rate)  # Downsample to 8 Hz
        
        # Normalize using Z-score normalization
        tr_data_mean = np.mean(tr.data)
        tr_data_std = np.std(tr.data)
        tr.data = (tr.data - tr_data_mean) / tr_data_std  # Z-score normalization
        
    #add data length to array
    length = np.append(length, len(tr.data))

    # Save the processed data
    output_file = os.path.join(processed_folder, os.path.basename(file_path))
    st.write(output_file, format="MSEED")
    print(f'processed file: {output_file}')

Check the lengths of the files.

In [3]:
# import os
# from obspy import read
# import glob
# import numpy as np

# Define the processed folder path = output_folder
processed_folder = os.getcwd() + '/data/processed'

# Array to store the lengths of each file
lengths = []

# Check each processed file's length
for file_path in glob.glob(f"{processed_folder}/*.mseed"):
    # Read only metadata
    st = read(file_path, headonly=True)
    
    # Extract the number of data points in each trace
    for tr in st:
        lengths.append(tr.stats.npts)

# Convert to numpy array for easier analysis
lengths = np.array(lengths)

# Check if all files have the same length
if np.all(lengths == lengths[0]):
    print(f"All files have the same length: {lengths[0]} data points.")
else:
    print("Files have different lengths:", lengths)

IndexError: index 0 is out of bounds for axis 0 with size 0

~~This is the correct number of points (+1) for 24 hours of data sampled at 25 Hz. The single extra point is a product of the decimation, will be removed in the preparing ai ready data notebook.~~